In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install pycaret

In [ ]:
%%time
# import big dtda
import datatable as dt
train = dt.fread('../input/tabular-playground-series-oct-2021/train.csv').to_pandas()
test = dt.fread('../input/tabular-playground-series-oct-2021/test.csv').to_pandas()
ss = dt.fread('../input/tabular-playground-series-oct-2021/sample_submission.csv').to_pandas()
print("train.shape",train.shape)
print("test.shape",test.shape)

In [ ]:
import random

data_ratio = 0.0001

sample_number=int(train.shape[0] * data_ratio)
idx=random.sample(range(train.shape[0]),sample_number)
train=train.iloc[idx]
print("number_of_training_dataset:",sample_number)

In [ ]:
import sys
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 10000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [ ]:
test.info()

In [ ]:
for c, dtype in zip(train.columns, train.dtypes):
    if dtype == np.float64:
        train[c] = train[c].astype(np.float32)
    elif dtype == np.int64:
        train[c] = train[c].astype(np.int32)

In [ ]:
for c, dtype in zip(test.columns, test.dtypes):
    if dtype == np.float64:
        test[c] = test[c].astype(np.float32)
    elif dtype == np.int64:
        test[c] = test[c].astype(np.int32)

In [ ]:
display(train.info())
display(test.info())

Now we minimized train and test data

In [ ]:
train.head()

In [ ]:
from pycaret.classification import *

In [ ]:
clf1 = setup(data = train, 
             target = 'target',
             ignore_low_variance = True,
             remove_multicollinearity = True,
             multicollinearity_threshold = 0.95,
             silent = True)

In [ ]:
# compare_models()

In [ ]:
%%time
catboost = create_model('catboost')

In [ ]:
%%time
tuned_cat = tune_model(catboost,optimize = 'AUC')

In [ ]:
print(tuned_cat.get_all_params())

In [ ]:
evaluate_model(tuned_cat)

In [ ]:
# finalize a model
cat_final = finalize_model(tuned_cat)

In [ ]:
%%time
gbc = create_model('gbc')

In [ ]:
%%time
tuned_gbc = tune_model(gbc,optimize = 'AUC')

In [ ]:
blender_specific = blend_models(estimator_list = [tuned_cat,tuned_gbc], method = 'soft')

In [ ]:
print(cat_final.best_score_)

# print(cat_final.best_params_)

In [ ]:
# generate predictions on unseen data
predictions = predict_model(blender_specific, data = test)

In [ ]:
predictions['Label']

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

submission['target'] = predictions['Label'].replace({'True':1,'False':0})
submission.to_csv("submission_Blend_OCT.csv", index=False)
submission

In [ ]:
# !echo "qid,prediction" > submission.csv

In [ ]:
from IPython.display import FileLink
FileLink('submission_Blend_OCT.csv')

In [ ]:
# %%time
# from IPython.display import HTML

# submission.to_csv('submission.csv', index=False)

# def create_download_link(title = "Download CSV file", filename = "data.csv"):  
#     html = '<a href={filename}>{title}</a>'
#     html = html.format(title=title,filename=filename)
#     return HTML(html)

# # create a link to download the dataframe which was saved with .to_csv method
# create_download_link(filename='submission.csv')